In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

In [2]:
#from factory.LstmDV import LstmDV as D_VECTOR
#from factory.MetaDV import MetaDV as D_VECTOR
from factory.deepspeach import DeepSpeaker as D_VECTOR
from util.Dataloader import generate_dataset

In [3]:
num_uttrs = 200
len_crop = 256
batch_size = 32
num_classes = 80
use_shuffle = True
isDeepSpeaker = True
save_name = "metadv_vctk80.pt"

In [4]:
d_vector = D_VECTOR(num_classes).to('cuda:0')
optimizer = torch.optim.Adam(d_vector.parameters(), 1e-4) 
criterion = nn.CrossEntropyLoss()

In [5]:
train_loader = generate_dataset(rootDir='train_spmel_vctk80',len_crop=len_crop,num_uttrs=num_uttrs, batch_size=batch_size,use_shuffle=use_shuffle)

Found directory: train_spmel_vctk80
Processing speaker: p225


In [6]:
epochs = 1

In [7]:
early_stop_count = 0
early_repeate = 3
last_loss = 100.0
for epoch in range(epochs):
    n_correct,n_total,train_loss,train_acc,counter = 0,0,0,0,0
    for data in train_loader:
        inputs, targets = data
        targets = targets.view(-1).long()
        if isDeepSpeaker:
            inputs = inputs.transpose(1,-1)
        current_sample = inputs.size(0)  
        output,_ = d_vector(inputs)
        n_correct += (torch.max(output, 1)[1].long().view(targets.size()) == targets).sum().item()
        n_total += current_sample
        train_acc_temp = 100. * n_correct / n_total
        loss = criterion(output, targets)
        train_loss += loss.item()
        train_acc += train_acc_temp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        counter += 1
    if epoch == 0:
        last_loss = (train_loss/counter)
    elif last_loss > (train_loss/counter):
        last_loss =  train_loss/counter
        early_stop_count = 0
    else:
        last_loss =  train_loss/counter
        early_stop_count += 1
        
        
    print(f"AVE LOSS --- {train_loss/counter}")
    print(f"ACC --- {train_acc/counter}")
    
    if early_stop_count == early_repeate:
        print("*************************")
        print(f" Training Stop at epoch --- {epoch}")
        print(f" Final AVE LOSS --- {train_loss/counter}")
        print(f" Final ACC --- {train_acc/counter}")
        print("*************************")
        break

AVE LOSS --- 3.1548349176134383
ACC --- 58.35267857142857


In [8]:
torch.save(d_vector,save_name)